In [ ]:
!pip install torch==1.9 torchtext==0.10

In [ ]:
!pip install datasets

     |████████████████████████████████| 245kB 2.9MB/s 
     |████████████████████████████████| 245kB 4.7MB/s 
     |████████████████████████████████| 122kB 5.2MB/s 


In [ ]:
import math
import time
import random

import torch
import torch.nn as nn
import torch.optim as optim

import torchtext

import datasets

In [ ]:
print(torch.__version__)
print(torchtext.__version__)

1.9.0+cu102
0.10.0


In [ ]:
torch.manual_seed(0)
random.seed(0)

In [ ]:
dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20. Subsequent calls will reuse this data.


In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [ ]:
dataset['train'][0]

{'text': ''}

In [ ]:
dataset['train'][1]

{'text': ' = Valkyria Chronicles III = \n'}

In [ ]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

In [ ]:
tokenizer('hello world how are you?')

['hello', 'world', 'how', 'are', 'you', '?']

In [ ]:
tokenizer(dataset['train'][1]['text'])

['=', 'valkyria', 'chronicles', 'iii', '=']

In [ ]:
def tokenize_data(example, tokenizer):
    tokens = {'tokens': tokenizer(example['text'])}
    return tokens

In [ ]:
tokenized_dataset = dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})

In [ ]:
tokenized_dataset['train'][1]

{'tokens': ['=', 'valkyria', 'chronicles', 'iii', '=']}

In [ ]:
vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['train']['tokens'],
                                                  min_freq=3)

In [ ]:
vocab.get_itos()[:10]

['the', ',', '.', 'of', 'and', 'in', 'to', 'a', '=', 'was']

In [ ]:
len(vocab)

29471

In [ ]:
'hello' in vocab

False

In [ ]:
vocab.insert_token('<unk>', 0)

In [ ]:
vocab.get_itos()[:10]

['<unk>', 'the', ',', '.', 'of', 'and', 'in', 'to', 'a', '=']

In [ ]:
vocab.set_default_index(0)

In [ ]:
vocab['hello']

0

In [ ]:
vocab.insert_token('<eos>', 1)

In [ ]:
vocab.get_itos()[:10]

['<unk>', '<eos>', 'the', ',', '.', 'of', 'and', 'in', 'to', 'a']

In [ ]:
def get_data(dataset, vocab, batch_size):
    data = []
    for example in dataset:
        if example['tokens']:
            tokens = example['tokens'].append('<eos>')
            tokens = [vocab[token] for token in example['tokens']]
            data.extend(tokens)
    data = torch.LongTensor(data)
    n_batches = data.shape[0] // batch_size
    data = data.narrow(0, 0, n_batches * batch_size)
    data = data.view(batch_size, -1)
    return data

In [ ]:
batch_size = 128

train_data = get_data(tokenized_dataset['train'], vocab, batch_size)

In [ ]:
train_data.shape

torch.Size([128, 16214])

In [ ]:
valid_data = get_data(tokenized_dataset['validation'], vocab, batch_size)
test_data = get_data(tokenized_dataset['test'], vocab, batch_size)

In [ ]:
class LockedDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, x):
        # x = [batch size, seq len, hidden dim]
        if not self.training or not self.p:
            return x
        x = x.clone()
        mask = x.new_empty(x.shape[0], 1, x.shape[2], requires_grad=False).bernoulli_(1 - self.p)
        mask = mask.div_(1 - self.p)
        mask = mask.expand_as(x)
        return x * mask

In [ ]:
def _setup_weight_drop(module, weights, dropout):
    for name_w in weights:
        w = getattr(module, name_w)
        del module._parameters[name_w]
        module.register_parameter(name_w + '_raw', nn.Parameter(w))

    original_module_forward = module.forward

    def forward(*args, **kwargs):
        for name_w in weights:
            raw_w = getattr(module, name_w + '_raw')
            w = nn.Parameter(torch.nn.functional.dropout(raw_w, p=dropout, training=module.training))
            setattr(module, name_w, w)

        return original_module_forward(*args, **kwargs)

    setattr(module, 'forward', forward)

In [ ]:
class WeightDropLSTM(torch.nn.LSTM):
    """
    Wrapper around :class:`torch.nn.LSTM` that adds ``weight_dropout`` named argument.

    Args:
        weight_dropout (float): The probability a weight will be dropped.
    """

    def __init__(self, *args, weight_dropout=0.0, **kwargs):
        super().__init__(*args, **kwargs)
        weights = ['weight_hh_l' + str(i) for i in range(self.num_layers)]
        _setup_weight_drop(self, weights, weight_dropout)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, 
                 embedding_dropout_rate, weight_dropout_rate, lstm_dropout_rate, output_dropout_rate, 
                 tie_weights):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        lstms = []
        for n in range(n_layers):
            input_dim = embedding_dim if n == 0 else hidden_dim
            lstm = WeightDropLSTM(input_dim, hidden_dim, batch_first=True, weight_dropout=weight_dropout_rate)
            lstms.append(lstm)
        self.lstms = nn.ModuleList(lstms)
        self.fc = nn.Linear(hidden_dim, vocab_size)

        self.embedding_dropout = LockedDropout(embedding_dropout_rate)
        self.lstm_dropout = LockedDropout(lstm_dropout_rate)
        self.output_dropout = LockedDropout(output_dropout_rate)

        if tie_weights:
            assert embedding_dim == hidden_dim, 'If tying weights then embedding_dim must equal hidden_dim'
            self.embedding.weight = self.fc.weight

        self.init_weights()
    
    def init_weights(self):
        init_range = 0.1
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()

    def init_hidden(self, batch_size, device):
        hiddens = []
        for n in range(self.n_layers):
            hidden = torch.zeros(1, batch_size, self.hidden_dim).to(device)
            cell = torch.zeros(1, batch_size, self.hidden_dim).to(device)
            hiddens.append((hidden, cell))
        return hiddens

    def detach_hidden(self, hidden):
        if isinstance(hidden, torch.Tensor):
            return hidden.detach()
        else:
            return tuple(self.detach_hidden(h) for h in hidden)

    def forward(self, input, hidden):
        # input = [batch size, seq len]
        # hidden = [n layers, batch size, hidden dim]
        embedding = self.embedding_dropout(self.embedding(input))
        # embedding = [batch size, seq len, embedding dim]
        lstm_input = embedding
        new_hiddens = []
        for n, lstm in enumerate(self.lstms):
            lstm_output, new_hidden = lstm(lstm_input, hidden[n])
            # lstm_output = [batch size, seq len, hidden dim]
            # new_hidden = [n layers, batch size, hidden dim]
            if n != self.n_layers - 1:
                lstm_output = self.lstm_dropout(lstm_output)
            lstm_input = lstm_output
            new_hiddens.append(new_hidden)
        output = self.output_dropout(lstm_output)
        output = self.fc(output)
        # output = [batch size, seq len, vocab size]
        return output, new_hiddens

In [ ]:
vocab_size = len(vocab)
embedding_dim = 1024
hidden_dim = 1024
n_layers = 2
embedding_dropout_rate = 0.4
weight_dropout_rate = 0.4
lstm_dropout_rate = 0.4
output_dropout_rate = 0.4
tie_weights = True

model = LSTM(vocab_size, embedding_dim, hidden_dim, n_layers,
             embedding_dropout_rate, weight_dropout_rate, lstm_dropout_rate, output_dropout_rate, 
             tie_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 47,003,425 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, data, optimizer, criterion, batch_size, base_seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    n_tokens = data.shape[-1]
    base_lr = optimizer.param_groups[0]["lr"]

    hidden = model.init_hidden(batch_size, device)
    
    for input, target in get_batches(data, base_seq_len):
        optimizer.zero_grad()
        input = input.to(device)
        target = target.to(device)
        # input = [batch size, seq len]
        # target = [batch size, seq len]
        batch_size, seq_len = input.shape
        scaled_lr = base_lr * seq_len / base_seq_len
        optimizer.param_groups[0]["lr"] = scaled_lr
        hidden = model.detach_hidden(hidden)
        # hidden = [n layers, batch size, hidden dim]
        output, hidden = model(input, hidden)
        # output = [batch size, seq len, vocab size]
        # hidden = [n layers, batch size, hidden dim]
        output = output.reshape(batch_size * seq_len, -1)
        target = target.reshape(-1)
        # output = [batch size * seq len, vocab size]
        # target = [batch size * seq len]
        loss = criterion(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / n_tokens

In [ ]:
def get_batches(data, seq_len):
    data_len = data.shape[-1]
    total_seq_len = 0
    sampled_seq_lens = []
    min_seq_len, max_seq_len = int(seq_len * 0.9), int(seq_len * 1.1)
    while total_seq_len < data_len:
        sampled_seq_len = random.randint(min_seq_len, max_seq_len)
        sampled_seq_lens.append(sampled_seq_len)
        total_seq_len += sampled_seq_len
    sampled_seq_lens = sampled_seq_lens[:-1]
    pos = 0
    for sampled_seq_len in sampled_seq_lens:
        input = data[:,pos:pos+sampled_seq_len]
        target = data[:,pos+1:pos+sampled_seq_len+1]
        pos += sampled_seq_len
        yield input, target

In [ ]:
def evaluate(model, data, criterion, batch_size, base_seq_len, device):

    epoch_loss = 0
    model.eval()
    n_tokens = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for input, target in get_batches(data, base_seq_len):
            input = input.to(device)
            target = target.to(device)
            # input = [batch size, seq len]
            # target = [batch size, seq len]
            batch_size, seq_len = input.shape
            hidden = model.detach_hidden(hidden)
            # hidden = [n layers, batch size, hidden dim]
            output, hidden = model(input, hidden)
            # output = [batch size, seq len, vocab size]
            # hidden = [n layers, batch size, hidden dim]
            output = output.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)
            # output = [batch size * seq len, vocab size]
            # target = [batch size * seq len]
            loss = criterion(output, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / n_tokens

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

In [ ]:
n_epochs = 50
seq_len = 50
clip = 0.25

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.monotonic()

    train_loss = train(model, train_data, optimizer, criterion, batch_size, seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, seq_len, device)
    
    lr_scheduler.step(valid_loss)

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'awd-lstm_lm.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:680: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:924.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


Epoch: 01 | Epoch Time: 1m 30s
	Train Perplexity: 599.371
	Valid Perplexity: 243.078
Epoch: 02 | Epoch Time: 1m 30s
	Train Perplexity: 260.736
	Valid Perplexity: 185.420
Epoch: 03 | Epoch Time: 1m 30s
	Train Perplexity: 185.362
	Valid Perplexity: 150.217
Epoch: 04 | Epoch Time: 1m 30s
	Train Perplexity: 141.389
	Valid Perplexity: 137.517
Epoch: 05 | Epoch Time: 1m 30s
	Train Perplexity: 115.362
	Valid Perplexity: 120.790
Epoch: 06 | Epoch Time: 1m 30s
	Train Perplexity: 99.415
	Valid Perplexity: 125.349
Epoch: 07 | Epoch Time: 1m 30s
	Train Perplexity: 85.013
	Valid Perplexity: 125.727
Epoch: 08 | Epoch Time: 1m 30s
	Train Perplexity: 79.302
	Valid Perplexity: 108.041
Epoch: 09 | Epoch Time: 1m 30s
	Train Perplexity: 77.275
	Valid Perplexity: 109.301
Epoch: 10 | Epoch Time: 1m 30s
	Train Perplexity: 75.031
	Valid Perplexity: 122.888
Epoch: 11 | Epoch Time: 1m 30s
	Train Perplexity: 74.193
	Valid Perplexity: 106.768
Epoch: 12 | Epoch Time: 1m 30s
	Train Perplexity: 74.398
	Valid Perplex

In [ ]:
model.load_state_dict(torch.load('awd-lstm_lm.pt'))

test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)

print(f'Test Perplexity: {math.exp(test_loss):.3f}')

NameError: ignored

In [ ]:
def generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed=None):

    if seed is not None:
        torch.manual_seed(0)

    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(n_gen_tokens):
            input = torch.LongTensor([indices]).to(device)
            output, hidden = model(input, hidden)
            probs = torch.softmax(output[:, -1] / temperature, dim=-1) 
            prediction = torch.multinomial(probs, num_samples=1).item()
            indices.append(prediction)

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [ ]:
prompt = 'my favourite color'
n_gen_tokens = 25
temperature = 0.1
seed = 0

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

In [ ]:
temperature = 0.25

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

In [ ]:
temperature = 0.75

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

In [ ]:
temperature = 1.0

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

In [ ]:
temperature = 2.5

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

In [ ]:
temperature = 5.0

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation

In [ ]:
temperature = 10.0

generation = generate(prompt, n_gen_tokens, temperature, model, tokenizer, vocab, device, seed)

In [ ]:
generation